In [ ]:
!pip install transformers
!pip install seqeval

     |████████████████████████████████| 1.1MB 4.3MB/s 
     |████████████████████████████████| 3.0MB 13.0MB/s 
     |████████████████████████████████| 1.1MB 46.3MB/s 
     |████████████████████████████████| 890kB 49.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=098a2c2008e97e5db1dce539a2a9244b2ed160790752eb7deba165e1810867ae
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for seqeval: filename=seqeval-0.0.19-cp36-none-any.whl size=9919 sha256=934644b4533bae2f14d91c662ec5fa1332fabcde3de67c049850a812c73e1522
  Stored in directory: /root/.cache/pip/wheels/8d/1f/bf/1198beceed805a2099060975f6281d1b01046dd279e19c97be
Successfully built seqeval


####We will be using hugging face stat-of-the are nlp mdoels with wrapper of tensorflow on it 

In [ ]:
! unzip /content/NLP_Assignment.zip

Archive:  /content/NLP_Assignment.zip
   creating: NLP Assignment/
  inflating: NLP Assignment/train.csv  
  inflating: NLP Assignment/test.csv  
   creating: NLP Assignment/train_data/
  inflating: NLP Assignment/train_data/11875_2017_Judgement_13-Nov-2017.txt  
  inflating: NLP Assignment/train_data/CRLREVP902005.txt  
  inflating: NLP Assignment/train_data/WA1342017.txt  
  inflating: NLP Assignment/train_data/WPC41342010.txt  
  inflating: NLP Assignment/train_data/WA1742017.txt  
  inflating: NLP Assignment/train_data/10210_2010_Judgement_06-Dec-2017.txt  
  inflating: NLP Assignment/train_data/10689_2008_Judgement_17-May-2018.txt  
  inflating: NLP Assignment/train_data/WPC13972016.txt  
  inflating: NLP Assignment/train_data/CRP3022016.txt  
  inflating: NLP Assignment/train_data/CrlPet8092014.txt  
  inflating: NLP Assignment/train_data/WPC45672011.txt  
  inflating: NLP Assignment/train_data/RevnPet42016.txt  
  inflating: NLP Assignment/train_data/WPC44562017.txt  
  inflatin

## Data prepration 

1.   CPU bound , I/O bounds programme fastening up the data prepation process

*   >threadpools for fast accessing the files store them in list of list 
*   >processpool for fast computation for the function

2. Format of data
* >metadata : sentence_id , doc_id , word , tag

3. Strategy 


1. > using threadpool programmes we will be accessing and reading each files and store them in hashmap data structure 
2. > using processpool programmes we will be accessing hashmap to process each each key value pairs and store them in final_csv for training the model
3. > cleanning of data is required over special chars to search in paragraph for getting the annotation done








In [ ]:
import pandas as pd
import concurrent.futures
import string
import numpy as np
import time 
import nltk.data 
import os
import re
from collections import defaultdict
from nltk.tokenize import regexp_tokenize 
import torch.nn.functional as F
import os

import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score

from tqdm import tqdm, trange

nltk.download("punkt")
# Loading PunktSentenceTokenizer using English pickle file 
sent_tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 5.5MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 6.7MB 15.7MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for boto3: filename=boto3-1.15.13-py2.py3-none-any.whl size=127861 sha256=a2c26a33b5d3a7ed009acba6469ae97890fc6680e5f8f88f861c0a07b9acc4ef
  Stored in directory: /root/.cache/pip/wheels/35/5a/eb/fe69289e84a5dddca3cceac736e265215b1eee4adc65c48c65
Successfully built boto3


In [ ]:
#reading of files storing it as hashmap

def read_files(filename):
  store_data={}
  with open(filename) as f:
    data=f.read().splitlines()
    store_data[filename]=data
    return store_data
start=time.time()

filenames_train=["/content/NLP Assignment/train_data/"+i for i in os.listdir("/content/NLP Assignment/train_data")]
filenames_test=["/content/NLP Assignment/test_data/"+i for i in os.listdir("/content/NLP Assignment/test_data")]
pool = concurrent.futures.ThreadPoolExecutor(max_workers=10)

train_data=pool.map(read_files,filenames_train)
train_data=list(train_data)

test_data=pool.map(read_files,filenames_test)
test_data=list(test_data)

end=time.time()
print("prepation function execution :", abs(end-start),"secs")
print("length of train_data files: ",len(train_data))
print("length of test_data files: ",len(test_data))


prepation function execution : 0.3264503479003906 secs
length of train_data files:  1030
length of test_data files:  231


#### Sentence tokenizer has issue making in our data due space and punctuations non-uniformity some judges names are not able catch so we will be using whole text and then move on with the same strategy we had earlier


In [ ]:
# process pool executor taking same time as 4 for loop so avoid that concurrent for time being 
# small cleaning function and lot more things can be done too 
def cleaning_txt(data,data_split=None):
  data_pairs=defaultdict(list)
  start=time.time()
  for filenames in data:
    for key , values in filenames.items():
      for lines in values:
        for line in sent_tokenizer.tokenize(lines):
          line=' '.join(line.split())
          data_pairs[key].append(line)
  end=time.time()
  print(data_split)
  print("prepation function execution :", abs(end-start),"secs")
  return data_pairs

data_train_pairs=cleaning_txt(train_data,"train")
data_test_pairs=cleaning_txt(test_data,"test")

train
prepation function execution : 5.64803409576416 secs
test
prepation function execution : 1.244990348815918 secs


#### Final stage of Data preparation part is started where we will making our desired format to train it on neural nets

### Strategy for final part
#### we will taking up train.csv and it's label data 
#### another hashmap we will be needing to perform this function 
#### names vs file.txt (to handle linear collision multikey-index dictionary)
#### will search in each key file sentence if we found that we will replace that string with empty value and tokenize the whole modified string. 
#### so that end output will be like the way we want.


#### Data engg stage 1 forming final dictionaries for train and test

In [ ]:
def key_file_pair(filename):
  df=pd.read_csv(filename)
  csv_key=defaultdict(list)
  for file_name,label in zip(df["File_Name"],df["Name"]):
    labels=label.split(",")
    csv_key['/content/NLP Assignment/train_data/'+file_name]=labels
  return csv_key

In [ ]:
csv_key_train=key_file_pair("/content/NLP Assignment/train.csv")

In [ ]:
def data_prepation_stage1(csv_key,data_pairs,data=None,csv_test_key=None):
  if data=="train":
    final_dictionary=defaultdict(list)
    for key , names in csv_key.items():
      if key in data_pairs:
        for name in names:
          contents=data_pairs[key]
          for content in contents:
            if name in content:
              final_dictionary[key].append((name,"Person"))
              content=content.replace(name,"")
              word_token=regexp_tokenize(content, "[\w']+")
              bigrams = list(nltk.ngrams(word_token,2))
              for grams in bigrams:
                final_dictionary[key].append((grams,"O")) 
            else:
              word_token=regexp_tokenize(content, "[\w']+")
              bigrams = list(nltk.ngrams(word_token,2))
              for grams in bigrams:
                final_dictionary[key].append((grams,"O"))
  else:
      final_dictionary=defaultdict(list)
      for key in csv_test_key:
        if key in data_pairs:
          contents=data_pairs[key]
          for content in contents:
            word_token=regexp_tokenize(content, "[\w']+")
            bigrams = list(nltk.ngrams(word_token,2))
            for grams in bigrams:
              final_dictionary[key].append(grams) 

  return final_dictionary

In [ ]:
final_dictionary_train=data_prepation_stage1(csv_key_train,data_train_pairs,"train")
csv_test=pd.read_csv("/content/NLP Assignment/test.csv")
csv_test_key_values="/content/NLP Assignment/test_data/"+csv_test["File_Name"]
final_dictionary_test=data_prepation_stage1(csv_key_train,data_test_pairs,"test",csv_test_key_values)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: generator 'ngrams' raised StopIteration
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: generator 'ngrams' raised StopIteration
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: generator 'ngrams' raised StopIteration


In [ ]:
final_dictionary_train['/content/NLP Assignment/train_data/10__Judgement_11-May-2018.txt']

[(('IN', 'THE'), 'O'),
 (('THE', 'SUPREME'), 'O'),
 (('SUPREME', 'COURT'), 'O'),
 (('COURT', 'OF'), 'O'),
 (('OF', 'INDIA'), 'O'),
 (('CIVIL', 'APPELLATE'), 'O'),
 (('APPELLATE', 'JURISDICTION'), 'O'),
 (('CIVIL', 'APPEAL'), 'O'),
 (('APPEAL', 'No'), 'O'),
 (('No', '3159'), 'O'),
 (('3159', 'OF'), 'O'),
 (('OF', '2004'), 'O'),
 (('COMMISSIONER', 'OF'), 'O'),
 (('OF', 'CENTRAL'), 'O'),
 (('CENTRAL', 'EXCISE'), 'O'),
 (('INDORE', 'APPELLANT'), 'O'),
 (('APPELLANT', 'S'), 'O'),
 (('M', 'S'), 'O'),
 (('S', 'GRASIM'), 'O'),
 (('GRASIM', 'INDUSTRIES'), 'O'),
 (('INDUSTRIES', 'LTD'), 'O'),
 (('THROUGH', 'ITS'), 'O'),
 (('ITS', 'SECRETARY'), 'O'),
 (('SECRETARY', 'RESPONDENT'), 'O'),
 (('RESPONDENT', 'S'), 'O'),
 (('C', 'A'), 'O'),
 (('Nos', '3455'), 'O'),
 (('3455', '2004'), 'O'),
 (('2004', '7272'), 'O'),
 (('7272', '2005'), 'O'),
 (('2005', '2982'), 'O'),
 (('2982', '2985'), 'O'),
 (('2985', '2005'), 'O'),
 (('2005', '2986'), 'O'),
 (('2986', '2005'), 'O'),
 (('7143', '2005'), 'O'),
 (('200

#### Data engg stage 2 where we will storing final dictionary of both train test in csv

In [ ]:
def final_csv(final_dictionary,data=None):
  if data=="train":
    data=pd.DataFrame(columns=["File_Name","word","token"])
    for file_key , values in final_dictionary.items():
      file_list=[]
      word_token=[]
      token_value=[]
      for value in values:
        if value[1]=="Person":
          file_list.append(file_key)
          word_token.append(value[0])
          token_value.append(value[1])
        else:
          bigram_data=[' '.join(w) for w in value]
          file_list.append(file_key)
          word_token.append(bigram_data[0])
          token_value.append(bigram_data[1])
      minor_frame=pd.DataFrame(columns=["File_Name","word","token"])
      minor_frame["File_Name"]=file_list
      minor_frame["word"]=word_token
      minor_frame["token"]=token_value
      data=data.append(minor_frame)
  else:
    data=pd.DataFrame(columns=["File_Name","word"])
    for file_key , values in final_dictionary.items():
      file_list=[]
      word_token=[]
      for value in values:
        bigram_data=[''.join(w) for w in value]
        file_list.append(file_key)
        word_token.append(bigram_data[0])
      minor_frame=pd.DataFrame(columns=["File_Name","word"])
      minor_frame["File_Name"]=file_list
      minor_frame["word"]=word_token
      data=data.append(minor_frame)
  return data



#### As we can see the lenght of sentence is really large more than 512 so we will scrape of sentences 20 units before "person" tag and after "person" tag to preserve the sentence (actually this error I got more block down but showing up here because I will be making function here to clean the data more)

In [ ]:
final_train_set=final_csv(final_dictionary_train,"train")
final_test_set=final_csv(final_dictionary_test,"test")

In [ ]:
import gc
gc.collect()

0

In [ ]:
final_train_set.to_csv("train.csv")
final_test_set.to_csv("test.csv")

## find files where person got tagged get their index values and scrape the rows from orginal set of train data 

In [ ]:
final_train_set=final_train_set.reset_index()
final_test_set=final_test_set.reset_index()

In [ ]:
final_test_set=final_test_set.drop("index",axis=1)
final_train_set=final_train_set.drop("index",axis=1)

In [ ]:
len(final_train_set.index)

3315428

In [ ]:
files_person=final_train_set[final_train_set["token"]=="Person"]

In [ ]:
files_person.head()

,File_Name,word,token
32222,/content/NLP Assignment/train_data/10000_2008_...,Arun Mishra,Person
33590,/content/NLP Assignment/train_data/10022_2008_...,Abhay Manohar Sapre,Person
36226,/content/NLP Assignment/train_data/10034_2016_...,A.M. Khanwilkar,Person
37720,/content/NLP Assignment/train_data/10034_2016_...,A.M. Khanwilkar,Person
45730,/content/NLP Assignment/train_data/10091_2003_...,R.K. Agrawal,Person


In [ ]:
person_index=files_person.index

In [ ]:
new_train=pd.DataFrame(columns=["File_Name","word","token"])
for i in person_index:
  new_data=final_train_set.iloc[i - 25 : i + 20]
  new_train=new_train.append(new_data)

In [ ]:
new_train.shape

(15333, 3)

# So as far we have prepared the data for training now we are moving with data preprocessing and training of our data 

# step1 .grouping of documents and their words with it's tag

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w,t) for w, t in zip(s["word"].values.tolist(),s["token"].values.tolist())]
        self.grouped = self.data.groupby("File_Name").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
# Get full document data structure
getter = SentenceGetter(new_train)

In [ ]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'and sentence sentence as as recorded recorded against against the the appellants appellants are are acquitted acquitted of of the the charge charge of of Section Section 498A 498A IPC IPC leveled leveled against The appellants appellants are are on on bail Their bail bail bonds bonds stand stand discharged Arun Mishra Uday Umesh Umesh Lalit New Delhi May 9 9 2018 Non Reportable IN THE THE SUPREME SUPREME COURT COURT OF OF INDIA CRIMINAL APPELLATE APPELLATE JURISDICTION CRIMINAL APPEAL APPEAL NO NO 1340 1340 OF OF 2013 Manoharan Anr'

In [ ]:
labels = [[s[1] for s in sent] for sent in getter.sentences]
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Person', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
tags_vals = list(set(new_train["token"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [ ]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

Make training data
Make raw data into trainable data for BERT, including:
Set gpu environment
Load tokenizer and tokenize
Set 3 embedding, token embedding, mask word embedding, segmentation embedding
Split data set into train and validate, then send them to dataloader

In [ ]:
#setup environment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
n_gpu

1

### Load tokenizer
### download the tokenizer file into local folder first 

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt

--2020-10-07 15:49:47--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.131.101
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.131.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 213450 (208K) [text/plain]
Saving to: ‘bert-base-cased-vocab.txt’

bert-base-cased-voc 100%[===================>] 208.45K  1016KB/s    in 0.2s    

2020-10-07 15:49:48 (1016 KB/s) - ‘bert-base-cased-vocab.txt’ saved [213450/213450]



In [ ]:
vocabulary = "/content/bert-base-cased-vocab.txt"

In [ ]:
max_len  = 176

In [ ]:
tokenizer=BertTokenizer(vocab_file=vocabulary,do_lower_case=False)

Tokenizer text

In hunggieface for bert, when come across OOV, will word piece the word
We need to adjust the labels base on the tokenize result, “##abc” need to set label "X"
Need to set "[CLS]" at front and "[SEP]" at the end, as what the paper do, BERT indexer should add [CLS] and [SEP] tokens

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 767662.03B/s]


In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(len(tokenized_texts[0]))

115


In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=176, dtype="long", truncating="post", padding="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=176, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [ ]:
# Since only one sentence, all the segment set to 0
segment_ids = [[0] * len(input_id) for input_id in input_ids]
segment_ids[0];

Split data into train and validate
70% for training, 30% for validation

Split all data

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(input_ids, tags,attention_masks,segment_ids, 
                                                            random_state=4, test_size=0.3)

set data to tensor

In [ ]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)


(231, 99, 231, 99)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy 

Put data into data loader



In [ ]:
batch_num = 32

In [ ]:
# Only set token embedding, attention embedding, no segment embedding for seqeunce tagging task segment-id is not needed
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

Train model

In [ ]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
# Set model to GPU,if you are using GPU machine
model.cuda();

In [ ]:
# Set epoch and grad max num
epochs = 50
max_grad_norm = 1.0

In [ ]:
import math


In [ ]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [ ]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
FULL_FINETUNING = True

In [ ]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

Fine-tuning model


In [ ]:
model.train()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 231
  Batch size = 32
  Num steps = 400


Epoch:   2%|▏         | 1/50 [00:06<05:12,  6.37s/it]

Train loss: 0.22720687304224288


Epoch:   4%|▍         | 2/50 [00:12<05:00,  6.27s/it]

...

Train loss: 0.0016342324504096592


Epoch:  98%|█████████▊| 49/50 [05:31<00:06,  6.90s/it]

Train loss: 0.0018988024343603424


Epoch: 100%|██████████| 50/50 [05:38<00:00,  6.77s/it]

Train loss: 0.0018893460801336914


In [ ]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

Save model

In [ ]:
bert_out_address = 'models/bert_out_model/judge20'
# Make dir if not exits
if not os.path.exists(bert_out_address):
        os.makedirs(bert_out_address)
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

'models/bert_out_model/judge20/vocab.txt'

# Load model

In [ ]:
model = BertForTokenClassification.from_pretrained(bert_out_address,num_labels=len(tag2idx))

In [ ]:
# Set model to GPU
model.cuda();

# Eval model


In [ ]:
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
#     if step > 2:
#         break
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

        

print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)

# Save the report into file
output_eval_file = os.path.join(bert_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =99
  Batch size = 32


/usr/local/lib/python3.6/dist-packages/seqeval/metrics/sequence_labeling.py:42: UserWarning: Person seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


f1 socre: 0.483721
Accuracy score: 0.989924
***** Eval results *****

              precision    recall  f1-score   support

       erson     0.5049    0.4643    0.4837       112

   micro avg     0.5049    0.4643    0.4837       112
   macro avg     0.5049    0.4643    0.4837       112
weighted avg     0.5049    0.4643    0.4837       112

f1 socre: 0.483721
Accuracy score: 0.989924


# Submission task

In [ ]:
final_test_set.head()

,File_Name,word
0,/content/NLP Assignment/test_data/MMD20052009W...,ORISSA
1,/content/NLP Assignment/test_data/MMD20052009W...,HIGH
2,/content/NLP Assignment/test_data/MMD20052009W...,COURT
3,/content/NLP Assignment/test_data/MMD20052009W...,W
4,/content/NLP Assignment/test_data/MMD20052009W...,C


Make query into embeddings
token id embedding, need to tokenize first
mask word embedding
segmentation embedding

In [ ]:
last_index=[]
new_test=pd.DataFrame(columns=["File_Name","word"])
for i in final_test_set["File_Name"].unique():
  new_data=final_test_set[final_test_set["File_Name"]==i]
  last_index.append(new_data.index[-1])

In [ ]:
for index,file_name in zip(last_index,final_test_set["File_Name"].unique()):
  new_data=final_test_set[final_test_set["File_Name"]==file_name]
  new_data=new_data.iloc[index - 600 : index]
  new_test=new_test.append(new_data)

In [ ]:
new_test

,File_Name,word
5676,/content/NLP Assignment/test_data/MMD20052009W...,than
5677,/content/NLP Assignment/test_data/MMD20052009W...,what
5678,/content/NLP Assignment/test_data/MMD20052009W...,has
5679,/content/NLP Assignment/test_data/MMD20052009W...,awarded
5680,/content/NLP Assignment/test_data/MMD20052009W...,to
...,...,...
2673,/content/NLP Assignment/test_data/CrlA2212011.txt,LCR
2674,/content/NLP Assignment/test_data/CrlA2212011.txt,JUDGE
2675,/content/NLP Assignment/test_data/CrlA2212011.txt,Appeal
2676,/content/NLP Assignment/test_data/CrlA2212011.txt,N


In [ ]:
from scipy.special import softmax
save_model_address = 'models/bert_out_model/judge20'

In [ ]:
save_model = BertForTokenClassification.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [ ]:
result_files={}
for file_name in final_test_set["File_Name"].unique():
  test_df=final_test_set[final_test_set["File_Name"]==file_name]
  out = ' '.join(test_df["word"])
  tokenized_texts = []
  temp_token = []
  temp_token.append('[CLS]')
  token_list = tokenizer.tokenize(out)
  for m,token in enumerate(token_list):
    temp_token.append(token)
  if len(temp_token) > max_len-1:
    temp_token= temp_token[:max_len-1]
  temp_token.append('[SEP]')
  tokenized_texts.append(temp_token)
  # Make text token into id
  input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                            maxlen=max_len, dtype="long", truncating="post", padding="post")
  # For fine tune of predict, with token mask is 1,pad token is 0
  attention_masks = [[int(i>0) for i in ii] for ii in input_ids]
  attention_masks[0]
  segment_ids = [[0] * len(input_id) for input_id in input_ids]
  segment_ids[0]
  input_ids = torch.tensor(input_ids)
  attention_masks = torch.tensor(attention_masks)
  segment_ids = torch.tensor(segment_ids)
  save_model.eval()
  # Get model predict result
  with torch.no_grad():
    outputs = save_model(input_ids, token_type_ids=None,
    attention_mask=None,)
    # For eval mode, the first result of outputs is logits
    logits = outputs[0]
  # Make logits into numpy type predict result
  # The predict result contain each token's all tags predict result
  predict_results = logits.detach().cpu().numpy()
  result_arrays_soft = softmax(predict_results[0])
  # Get each token final predict tag index result
  result_list = np.argmax(result_arrays_soft,axis=-1)
  for i, mark in enumerate(attention_masks[0]):
    if mark>0:
      if tag2name[result_list[i]] == "Person":
        result_files[file_name]=temp_token[i]
        #print("Posibility:%f"%(result_array[i][result_list[i]]))

In [ ]:
submission_df=pd.DataFrame(columns=["File_Name","Name"])
files=[]
names=[]
for key , person_name in result_files.items():
  files.append(key)
  names.append(person_name)

In [ ]:
submission_df["File_Name"]=files
submission_df["Name"]=names

In [ ]:
submission_df.to_csv("submission.csv")